In [ ]:
import os
import sys

import pandas as pd
import altair as alt

cur_dir = os.getcwd()
SRC_PATH = cur_dir[
    : cur_dir.index("customer_complaint_analyzer") + len("customer_complaint_analyzer")
]
if SRC_PATH not in sys.path:
    sys.path.append(SRC_PATH)

from src.data.load_preprocess_data import load_raw_complaints_data

alt.data_transformers.enable('data_server')
alt.renderers.enable('mimetype')

In [ ]:
data_path = os.path.join(os.pardir, "data", "raw", "complaints.csv")

complaints_df = load_raw_complaints_data(data_path, num_rows=1_000_000)

In [ ]:
complaints_df.sort_values(by='date_received', ascending = False, inplace=True)

# complaints_df.company_public_response.fillna('None Recorded', inplace=True)
# complaints_df.sub_issue.fillna('None', inplace=True)
# complaints_df.consumer_complaint_narrative.fillna("None", inplace=True)

In [ ]:
complaints_df.head()

In [ ]:
complaints_df.info()

In [ ]:
complaints_df.describe()

# Missing Values

In [ ]:
complaints_df[:10].style.highlight_null()

In [ ]:
num_complaints = 2000
na_val_df = complaints_df.head(num_complaints).isna().reset_index().melt(
        id_vars='index'
    )
last_date = complaints_df.date_received.head(num_complaints).max().strftime("%m/%d/%Y")
first_date = complaints_df.date_received.head(num_complaints).min().strftime("%m/%d/%Y")

missing_vals = alt.Chart(
    complaints_df.head(num_complaints).isna().reset_index().melt(
        id_vars='index'
    ),
    title = f"Missing Values of {num_complaints} Complaints: {first_date} - {last_date}"
).mark_rect().encode(
    alt.X('index:O', axis=None),
    alt.Y('variable', title=None),
    alt.Color('value', title='Missing Value',scale=alt.Scale(scheme='dark2')),
    alt.Stroke('value', scale=alt.Scale(scheme='dark2'))  # We set the stroke which is the outline of each rectangle in the heatmap
).properties(
    width=min(1000, complaints_df.head(num_complaints).shape[0])
);
display(missing_vals)

## Insights
- from July 2022 to November no consumers were recorded as disputing a claim, potentially because they haven't been processed yet?
- what about older claims?

In [ ]:
na_val_df = complaints_df.tail(num_complaints).isna().reset_index().melt(
        id_vars='index'
    )
last_date = complaints_df.date_received.tail(num_complaints).max().strftime("%m/%d/%Y")
first_date = complaints_df.date_received.tail(num_complaints).min().strftime("%m/%d/%Y")

missing_vals = alt.Chart(
    complaints_df.tail(num_complaints).isna().reset_index().melt(
        id_vars='index'
    ),
    title = f"Missing Values of {num_complaints} Complaints: {first_date} - {last_date}"
).mark_rect().encode(
    alt.X('index:O', axis=None),
    alt.Y('variable', title=None),
    alt.Color('value', title='Missing Value',scale=alt.Scale(scheme='dark2')),
    alt.Stroke('value', scale=alt.Scale(scheme='dark2'))  # We set the stroke which is the outline of each rectangle in the heatmap
).properties(
    width=min(1000, complaints_df.tail(num_complaints).shape[0])
);
display(missing_vals)

## Insights - 

# Distribution of Categorical Variables

In [ ]:
complaints_df.columns

In [ ]:
max_bars = 10

columns = ["product", "sub_product", "issue", "sub_issue",
            "company_public_response", "company", "state", "consumer_consent_provided",
            "timely_response", "consumer_disputed"]

for col in columns:

    counts = complaints_df.groupby(col, dropna=False).size().reset_index(name='counts')
    count_bar = alt.Chart(
        counts,
        title = f"Most Common {col} Reported"
    ).mark_bar().encode(
        x='counts',
        y=alt.Y(col, type = "nominal", sort = "x"),
    ).transform_window(
        rank='rank(counts)',
        sort=[alt.SortField('counts', order='descending')]
    ).transform_filter(
        (alt.datum.rank <= max_bars)
    ).properties(
        width = 600
    )
    display(count_bar)
    


## Complaints Over Time

In [ ]:
num_complaints = complaints_df.resample("M", on="date_received").agg({'date_received': 'size'}).rename(columns={"date_received":"num_complaints"}).reset_index()

alt.Chart(
    num_complaints,
    title = "Monthly Complaints are Spiking in 2022"
).mark_line().encode(
	x = alt.X('date_received:T', title="Date Complaints Received"),
	y = alt.Y("num_complaints:Q", title = "No. of Monthly Complaints") 
).properties(
    width = 800,
    height = 400
)

## Complaints by Product

In [ ]:
proc_complaints_df = complaints_df.copy(deep=True)
proc_complaints_df["year_month"] = proc_complaints_df.date_received.apply(lambda dt: dt.replace(day=1))

In [ ]:
alt.Chart(
    proc_complaints_df.groupby(['year_month', "product"], as_index=False).agg(
        num_complaints = ("product", "size")
    ),
    title = "Credit Reporting & Credit Repairs Service Complaints are Majority of Complaints"
).mark_line().encode(
	x = alt.X('year_month:T', title="Date Complaints Received"),
	y = alt.Y("num_complaints", title = "No. of Monthly Complaints"),
    color = "product",
    tooltip="product"
).properties(
    width = 800,
    height = 400
).interactive()

In [ ]:
most_complaint_types = (
    proc_complaints_df.groupby(["product"], as_index=False)
    .agg(num_complaints = ("product", "size"))
    .sort_values(by = "num_complaints", ascending=False)
    .head(6)["product"]
    .to_list()
)

most_complaint_types = list(filter(lambda x: not x.startswith("Credit reporting, credit repair"), most_complaint_types))

alt.Chart(
    proc_complaints_df.loc[proc_complaints_df["product"].isin(most_complaint_types), :].groupby(['year_month', "product"], as_index=False).agg(
    num_complaints = ("product", "size")
    ),
    title = "Top 5 Most Common Products (Excluding Credit Reporting & Credit Repair)"
).mark_line().encode(
	x = alt.X('year_month:T', title="Date Complaints Received"),
	y = alt.Y("num_complaints", title = "No. of Monthly Complaints"),
    color = "product",
    tooltip="product"
).properties(
    width = 800,
    height = 400
).interactive()

In [ ]:
least_complaint_types = (
    proc_complaints_df.groupby(["product"], as_index=False)
    .agg(num_complaints = ("product", "size"))
    .sort_values(by = "num_complaints", ascending=True)
    .head(5)["product"]
    .to_list()
)

alt.Chart(
    proc_complaints_df.loc[proc_complaints_df["product"].isin(least_complaint_types), :].groupby(['year_month', "product"], as_index=False).agg(
    num_complaints = ("product", "size")
    ),
    title = "Top 5 Least Common Products with Complaints"
).mark_line().encode(
	x = alt.X('year_month:T', title="Date Complaints Received"),
	y = alt.Y("num_complaints", title = "No. of Monthly Complaints"),
    color = "product",
    tooltip="product"
).properties(
    width = 800,
    height = 400
).interactive()

## Complaints by Company

In [ ]:
alt.Chart(
    proc_complaints_df.groupby(['year_month', "company"], as_index=False).agg(
        num_complaints = ("company", "size")
    ),
    title = "Majority of Complaints are from TransUnion, Experian & Equifax"
).mark_line().encode(
	x = alt.X('year_month:T', title="Date Complaints Received"),
	y = alt.Y("num_complaints", title = "No. of Monthly Complaints"),
    color = "company",
    tooltip="company"
).properties(
    width = 800,
    height = 400
).interactive()

In [ ]:
most_complaint_types = (
    proc_complaints_df.groupby(["company"], as_index=False)
    .agg(num_complaints = ("company", "size"))
    .sort_values(by = "num_complaints", ascending=False)
    .head(6)["company"]
    .to_list()
)

alt.Chart(
    proc_complaints_df.loc[proc_complaints_df["company"].isin(most_complaint_types), :].groupby(['year_month', "company"], as_index=False).agg(
    num_complaints = ("company", "size")
    ),
    title = "Top 5 Most Common Companys (Excluding Credit Reporting & Credit Repair)"
).mark_line().encode(
	x = alt.X('year_month:T', title="Date Complaints Received"),
	y = alt.Y("num_complaints", title = "No. of Monthly Complaints"),
    color = "company",
    tooltip="company"
).properties(
    width = 800,
    height = 400
).interactive()

In [ ]:
least_complaint_types = (
    proc_complaints_df.groupby(["company"], as_index=False)
    .agg(num_complaints = ("company", "size"))
    .sort_values(by = "num_complaints", ascending=False)
    .head(15)["company"]
    .iloc[3:11]
    .to_list()
)

alt.Chart(
    proc_complaints_df.loc[proc_complaints_df["company"].isin(least_complaint_types), :].groupby(['year_month', "company"], as_index=False).agg(
    num_complaints = ("company", "size")
    ),
    title = "Next 8 Most Common Companies with Complaints"
).mark_line().encode(
	x = alt.X('year_month:T', title="Date Complaints Received"),
	y = alt.Y("num_complaints", title = "No. of Monthly Complaints"),
    color = "company",
    tooltip="company"
).properties(
    width = 800,
    height = 400
).interactive()

## Top 3 Companies with Most Complaints: Equifax, TransUnion, Experian

In [ ]:
top_companies = ["EQUIFAX, INC.", "TRANSUNION INTERMEDIATE HOLDINGS, INC.", "Experian Information Solutions Inc."]

top_comp_df = proc_complaints_df.loc[proc_complaints_df["company"].isin(top_companies), :]


In [ ]:
max_bars = 10

columns = ["product", "sub_product", "issue", "sub_issue",
            "company_public_response", "company", "state", "consumer_consent_provided",
            "timely_response", "consumer_disputed"]

for col in columns:

    counts = top_comp_df.groupby(col, dropna=False).size().reset_index(name='counts')
    count_bar = alt.Chart(
        counts,
        title = f"Most Common {col} Reported"
    ).mark_bar().encode(
        x='counts',
        y=alt.Y(col, type = "nominal", sort = "x"),
    ).transform_window(
        rank='rank(counts)',
        sort=[alt.SortField('counts', order='descending')]
    ).transform_filter(
        (alt.datum.rank <= max_bars)
    ).properties(
        width = 600
    )
    display(count_bar)

In [ ]:
perc_timely = top_comp_df.timely_response.value_counts()[1] / top_comp_df.timely_response.value_counts()[0]
print(f"Top 3 Companies have {(1 - perc_timely)*100:.2f}% Timely Responses since {top_comp_df.date_received.min().year}")

In [ ]:
print(f"Top 3 Companies have had {top_comp_df.consumer_disputed.value_counts()[1]:.0f} Complaints disputed by Customers since {top_comp_df.date_received.min().year}")